In [1]:
import json
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt

In [2]:
def check_avg(df_, season, team, stats = ['AST']):
    avg_stats = ['AVG_'+stat for stat in stats]
    df = df_.loc[(df_['SEASON_ID'] == season)& (df_['TEAM_ABBREVIATION'] == team)].sort_values('GAME_DATE')
    avg = df[stats].expanding().mean().shift(1)
    diff = df[avg_stats].values - avg[stats].values
    return(np.max(diff[1:]), np.min(diff[1:]))

In [3]:
STATS = ['PTS', 'FG2M', 'FG2A', 'FG2_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'DREB', 'OREB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
AVG_STATS = ['AVG_'+stat for stat in STATS]
OPP_AVG_STATS = ['OPP_'+stat for stat in AVG_STATS]

In [26]:
df = pd.read_csv('dat/1983-2017.csv')
df = df.set_index('GAME_ID').drop(columns='Unnamed: 0').sort_values('GAME_DATE')
df['FG3_PCT'] = df['FG3_PCT'].fillna(0)
df['FG3A'] = df['FG3A'].fillna(0)
df['PLUS_MINUS'] = df['PTS'] - df['PTS_A']
df = df.reset_index()

In [27]:
df.sample(10)

,GAME_ID,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS_A,FG2M,FG2A,FG2_PCT,WINS,PTS_AVG,PTS_A_AVG,TOTAL_GAMES,TOTAL_WINS,GP,PYE,PYE_WINS,PYE_WINS_ERROR,PYE_WINS_ERROR_ABS,PYE_PROJ,PYE_PROJ_ERROR,PYE_PROJ_ERROR_ABS,year
38622,20100986,22001,1610612765,DET,Detroit Pistons,2002-03-22,DET @ LAL,L,241,82,26,69,0.377,7,19.0,0.368,23,34,0.676,8.0,28.0,36.0,18,10.0,1.0,20.0,18.0,-12,94,19,50.0,0.380000,40.0,93.264706,91.705882,82,50.0,68,0.558347,37.967597,2.032403,2.032403,47.816858,2.183142,2.183142,2001-02
10851,28800712,21988,1610612758,SAC,Sacramento Kings,1989-03-05,SAC vs. GOS,W,240,110,46,108,0.426,7,18.0,0.389,11,18,0.611,23.0,41.0,64.0,23,4.0,12.0,13.0,23.0,14,96,39,90.0,0.433333,16.0,104.379310,111.362069,82,27.0,58,0.288897,16.756037,-0.756037,0.756037,22.933532,4.066468,4.066468,1988-89
38871,20101111,22001,1610612746,LAC,Los Angeles Clippers,2002-04-08,LAC @ SEA,L,241,100,40,70,0.571,3,6.0,0.500,17,28,0.607,10.0,16.0,26.0,21,8.0,6.0,13.0,21.0,-2,102,37,64.0,0.578125,38.0,95.474359,95.730769,82,39.0,78,0.490674,38.272593,-0.272593,0.272593,39.962697,-0.962697,0.962697,2001-02
64223,21200626,22012,1610612758,SAC,Sacramento Kings,2013-01-23,SAC vs. PHX,L,240,96,38,79,0.481,5,11.0,0.455,15,24,0.625,18.0,31.0,49.0,23,8.0,9.0,24.0,22.0,-10,106,33,68.0,0.485294,16.0,97.731707,103.463415,82,30.0,41,0.311577,12.774639,3.225361,3.225361,28.774639,1.225361,1.225361,2012-13
45659,20400932,22004,1610612737,ATL,Atlanta Hawks,2005-03-14,ATL vs. DET,L,265,108,46,89,0.517,2,8.0,0.250,14,15,0.933,11.0,27.0,38.0,23,7.0,2.0,11.0,24.0,-6,114,44,81.0,0.543210,11.0,91.507937,100.523810,82,13.0,63,0.212971,13.417190,-2.417190,2.417190,15.046454,-2.046454,2.046454,2004-05
19111,29200489,21992,1610612765,DET,Detroit Pistons,1993-01-22,DET @ SAN,L,240,109,45,95,0.474,2,7.0,0.286,17,19,0.895,13.0,26.0,39.0,14,5.0,4.0,11.0,30.0,-14,123,43,88.0,0.488636,17.0,99.236842,101.236842,82,40.0,38,0.431054,16.380055,0.619945,0.619945,35.966379,4.033621,4.033621,1992-93
6830,28600588,21986,1610612749,MIL,Milwaukee Bucks,1987-02-18,MIL @ UTH,W,240,113,46,90,0.511,1,8.0,0.125,20,24,0.833,11.0,25.0,36.0,29,14.0,0.0,18.0,23.0,4,109,45,82.0,0.548780,34.0,108.654545,105.127273,82,50.0,55,0.612790,33.703450,0.296550,0.296550,50.545330,-0.545330,0.545330,1986-87
49875,20600583,22006,1610612753,ORL,Orlando Magic,2007-01-19,ORL vs. WAS,L,236,93,36,77,0.468,5,17.0,0.294,16,21,0.762,7.0,31.0,38.0,21,3.0,5.0,18.0,21.0,-21,114,31,60.0,0.516667,22.0,93.575000,92.525000,82,40.0,40,0.539161,21.566442,0.433558,0.433558,44.644764,-4.644764,4.644764,2006-07
20645,29300157,21993,1610612765,DET,Detroit Pistons,1993-11-27,DET @ NYK,L,240,85,32,80,0.400,4,11.0,0.364,17,20,0.850,11.0,23.0,34.0,21,5.0,3.0,18.0,22.0,-27,112,28,69.0,0.405797,5.0,101.090909,105.000000,82,20.0,11,0.371042,4.081467,0.918533,0.918533,31.344011,-11.344011,11.344011,1993-94
55120,20800745,22008,1610612743,DEN,Denver Nuggets,2009-02-07,DEN @ NJN,L,241,70,26,73,0.356,4,15.0,0.267,14,23,0.609,6.0,24.0,30.0,15,7.0,4.0,16.0,27.0,-44,114,22,58.0,0.379310,34.0,104.039216,100.784314,82,54.0,51,0.608767,31.047111,2.952889,2.952889,52.871773,1.128227,1.128227,2008-09


In [28]:
df[AVG_STATS] = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_ID']).groupby(['SEASON_ID', 'TEAM_ID'])[STATS].expanding().mean().reset_index((0,1))[STATS]
df[AVG_STATS] = df.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS].shift(1)
df = df.reset_index().sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_ID']).set_index('GAME_ID')

In [30]:
df.head(10)[['PTS', 'AVG_PTS', 'AST', 'AVG_AST']]

,PTS,AVG_PTS,AST,AVG_AST
GAME_ID,,,,
28300005,108,NaN,12,NaN
28300014,117,108.000000,24,12.000000
28300027,95,112.500000,17,18.000000
28300041,103,106.666667,25,17.666667
28300058,84,105.750000,18,19.500000
28300060,90,101.400000,15,19.200000
28300080,93,99.500000,15,18.500000
28300096,105,98.571429,14,18.000000
28300101,107,99.375000,24,17.500000


In [31]:
df['perc_complete'] = (df['GP'].values-1) / df['TOTAL_GAMES']

In [32]:
home = df.loc[~df['MATCHUP'].str.contains('@')].sort_index()
away = df.loc[df['MATCHUP'].str.contains('@')].sort_index()

In [33]:
home['HOME'] = 1
away['HOME'] = 0

In [34]:
opp_headers = ['OPP_'+stat for stat in STATS+AVG_STATS]

In [35]:
away_opp = away.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})
home_opp = home.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})

In [36]:
data = pd.concat([pd.concat([home, away_opp[opp_headers]], axis=1), pd.concat([away, home_opp[opp_headers]], axis=1)])

In [37]:
data['W'] = data['WL'].map({'W': 1, 'L': 0})

In [38]:
with open('dat/locations.json') as f:
    locations = json.loads(f.read())
data['LOCATION'] = data['TEAM_NAME'].map(locations)
data['LOCATION_W_PCT'] = data['LOCATION'].map(data.loc[data['HOME'] == 1].groupby('LOCATION')['W'].mean().to_dict())

In [39]:
data = data.reset_index()

In [43]:
data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).head()[['PTS', 'AVG_PTS', 'OPP_PTS', 'OPP_AVG_PTS']]

,PTS,AVG_PTS,OPP_PTS,OPP_AVG_PTS
60612,108,NaN,126,NaN
21748,117,108.000000,115,127.000000
21761,95,112.500000,92,105.500000
21775,103,106.666667,90,100.666667
60665,84,105.750000,97,101.500000


*NOTE: OPP_AVG_PTS is not average points scored against, it is AVG_PTS for other team*

In [15]:
data.to_csv('dat/expanding_avg_1983-2017.csv')

In [17]:
check_avg(data, 22017, 'PHI')

(0.0, 0.0)